In [1]:
%matplotlib notebook
from ipywidgets import *
from math import sqrt
import numpy as np
import matplotlib.pyplot as plt
import os.path
import re
import json
import numpy as np
from numpy import genfromtxt
import pandas as pd
from keras.layers import Dense
from keras.layers import LSTM
from keras.models import Sequential
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
import tensorflow as tf
from stldecompose import decompose, forecast
from stldecompose.forecast_funcs import (naive,
                                         drift, 
                                         mean, 
                                         seasonal_naive)
from pmdarima.arima import auto_arima
from sklearn import preprocessing
from rpy2.robjects.packages import STAP
import time
tf.logging.set_verbosity(tf.logging.ERROR)
import rpy2
from rpy2.rinterface_lib.callbacks import logger as rpy2_logger
import logging
rpy2_logger.setLevel(logging.ERROR)   # will display errors, but not warnings

Using TensorFlow backend.


In [2]:
DATASET_STATES_CSV = 'States_Timeseries.csv'
FOUR_THETA_FILE = 'four_theta.r'
THETA_FILE = 'theta.r'
ETS_FILE = 'ets.r'
TBATS_FILE = 'tbats.r'
BAGGED_ETS_FILE = 'bagged_ets.r'

LOOK_BACK = 12
FORECASTING_STEPS = 6
RANDOM_SEED = 3

SCALER_STANDARD = preprocessing.StandardScaler
SCALER_MINMAX = preprocessing.MinMaxScaler
DATA_SCALER = SCALER_MINMAX
EPOCHS = 50
BATCH_SIZE = 1

In [3]:
def normalize_df(df,scaler):
    x = df.values #returns a numpy array
    x_train = df.values[:-6]
    display(x_train.shape)
    scaler = scaler.fit(x_train)
    x_scaled = scaler.transform(x)
    df = pd.DataFrame(x_scaled, columns=df.columns, index=df.index)
    return df

def normalizeSeries(df,scaler):
    x = np.array(df.values)
    x = np.reshape(x,(-1,1))
    x = scaler.fit_transform(x)
    x = x[:,0]
    return x

def normalizeInputTouple(x,y,scaler):
    x = np.array(x)
    x = np.reshape(x,(-1,1))
    x = scaler.fit_transform(x)
    x = x[:,0]
    y = y.reshape(-1,1)
    y = scaler.transform(y)
    y = y[:,0]
    return x,y

def getTrainTestSplit(series):
    series_np = np.array(series)
    return series_np[:-FORECASTING_STEPS],series_np[-FORECASTING_STEPS:]

def reshapeInputs(data):
    data = np.array(data)
    return np.reshape(data, (data.shape[0], data.shape[1],1))

def create_dataset(dataset,look_back):
    dataX, dataY = [], []
    for i in range(0,len(dataset) - look_back -FORECASTING_STEPS+1, int(look_back/2)):
        a = dataset[i:(i + look_back)]
        dataX.append(a)
        dataY.append(dataset[i + look_back:i+look_back+FORECASTING_STEPS])
    return reshapeInputs(np.array(dataX)), np.array(dataY)

def create_dataset_scaled(dataset,scaler):
    dataX, dataY = [], []
    for i in range(0,len(dataset) - LOOK_BACK -FORECASTING_STEPS+1, int(LOOK_BACK/2)):
        x = dataset[i:(i + LOOK_BACK)]
        y = dataset[i + LOOK_BACK:i+LOOK_BACK+FORECASTING_STEPS]
        normalizeInputTouple(x,y,scaler)
        dataX.append(x)
        dataY.append(y)
    return reshapeInputs(dataX), np.array(dataY)

def getBaselineLSTM():
    model = Sequential()
    model.add(LSTM(16, input_shape=(LOOK_BACK, 1),return_sequences=True))
    model.add(LSTM(8))
    model.add(Dense(FORECASTING_STEPS, activation='sigmoid'))
    model.compile(loss='mse', optimizer='adam', metrics=['mean_absolute_error'])
    return model


def evaluateModel(model,x,y,scaler,wsc = False):
    prediction = model.predict(x)
    if not scaler == None:
        prediction = scaler.inverse_transform(prediction)
        if not wsc:
            y = scaler.inverse_transform(y)
    score = mean_squared_error(y.ravel(),prediction.ravel())
    return score

def evaluateLSTM(train,test,scaler):
    x_test = reshapeInputs([train[-LOOK_BACK:]])
    y_test = np.array([test])
    x_train,y_train = create_dataset(train,LOOK_BACK)
    model = getBaselineLSTM()
    model.fit(x_train, y_train, epochs=EPOCHS, batch_size=BATCH_SIZE, verbose=0)
    return evaluateModel(model,x_test,y_test,scaler)

def evaluateLSTM_windowScaled(train,test,scaler):
    x_train,y_train = create_dataset_scaled(train,scaler)
    model = getBaselineLSTM()
    model.fit(x_train, y_train, epochs=EPOCHS, batch_size=BATCH_SIZE, verbose=0)
    x_test =train[-LOOK_BACK:]
    y_test = test
    x_test,y_test = normalizeInputTouple(x_test,y_test,scaler)
    x_test = reshapeInputs([x_test])
    y_test = np.array([test])
    return evaluateModel(model,x_test,y_test,scaler,True)

def evaluateARIMAprediction(pred,y,scaler):
    pred = np.reshape(pred,(1,-1))
    y = np.reshape(y,(1,-1))
    if not scaler == None:
        pred = scaler.inverse_transform(pred)
        y = scaler.inverse_transform(y)
    return mean_squared_error(pred.ravel(), y.ravel())

def evaluateARMA(single_x,single_y,scaler):
    model = auto_arima(single_x, start_p=0, start_q=0, start_P=0, start_Q=0,
                 max_p=5, max_q=5, max_P=5, max_Q=5, seasonal=False,
                 stepwise=True, suppress_warnings=True, D=0, max_D=0,
                 error_action='ignore')
    pred = model.predict(n_periods=single_y.shape[0])
    return evaluateARIMAprediction(pred,single_y,scaler)

def evaluateARIMA(single_x,single_y,scaler):
    model = auto_arima(single_x, start_p=0, start_q=0, start_P=0, start_Q=0,
                 max_p=5, max_q=5, max_P=0, max_Q=0, seasonal=False,
                 stepwise=True, suppress_warnings=True, D=0, max_D=10,
                 error_action='ignore')
    pred = model.predict(n_periods=single_y.shape[0])
    return evaluateARIMAprediction(pred,single_y,scaler)

def evaluateSARIMA(single_x,single_y,scaler):
    model = auto_arima(single_x, start_p=0, start_q=0, start_P=0, start_Q=0,
                 max_p=5, max_q=5, max_P=5, max_Q=5, seasonal=True, m = 12,
                 stepwise=True, suppress_warnings=True, D=0, max_D=10,
                 error_action='ignore')
    pred = model.predict(n_periods=single_y.shape[0])
    return evaluateARIMAprediction(pred,single_y,scaler)

def getSARIMAPred(single_x):
    model = auto_arima(single_x, start_p=0, start_q=0, start_P=0, start_Q=0,
                 max_p=5, max_q=5, max_P=5, max_Q=5, seasonal=True, m = 12,
                 stepwise=True, suppress_warnings=True, D=0, max_D=10,
                 error_action='ignore')
    pred = model.predict(n_periods=6)
    return pred

def evaluateSTLSeasNaive(single_x,single_y,scaler):
    # apply the decomp to the truncated observation
    ind = pd.date_range('2008-01-01', periods=len(single_x), freq='M')
    single_x = pd.DataFrame(single_x,index=ind)
    short_decomp = decompose(single_x, period=12)
    pred = forecast(short_decomp, steps=6, fc_func=seasonal_naive).values
    return evaluateARIMAprediction(pred,single_y,scaler)




def evaluateFourTheta(single_x,single_y,scaler):
    with open(FOUR_THETA_FILE, 'r') as f:
        string = f.read()
    four_theta = STAP(string, "four_theta")
    res = four_theta.FourTheta(rpy2.robjects.FloatVector(single_x),rpy2.robjects.FloatVector([FORECASTING_STEPS]))
    pred = np.array(res[1])
    #display(res[1])
    return evaluateARIMAprediction(pred,single_y,scaler)

def evaluateTheta(single_x,single_y,scaler):
    with open(THETA_FILE, 'r') as f:
        string = f.read()
    theta = STAP(string, "theta")
    res = theta.thetaf(rpy2.robjects.FloatVector(single_x),rpy2.robjects.FloatVector([FORECASTING_STEPS]))
    pred = np.array(res[1])
    #display(res[1])
    return evaluateARIMAprediction(pred,single_y,scaler)

def getThetaPred(single_x):
    with open(THETA_FILE, 'r') as f:
        string = f.read()
    theta = STAP(string, "theta")
    res = theta.thetaf(rpy2.robjects.FloatVector(single_x),rpy2.robjects.FloatVector([FORECASTING_STEPS]))
    pred = np.array(res[1])
    #display(res[1])
    return pred

def evaluateETS(single_x,single_y,scaler):
    with open(ETS_FILE, 'r') as f:
        string = f.read()
    ets = STAP(string, "ets")
    res = ets.runets(rpy2.robjects.FloatVector(single_x),rpy2.robjects.FloatVector([FORECASTING_STEPS]))
    pred = np.array(res[1])
    #display(res[1])
    return evaluateARIMAprediction(pred,single_y,scaler)

def evaluateTBATS(single_x,single_y,scaler):
    with open(TBATS_FILE, 'r') as f:
        string = f.read()
    tbats = STAP(string, "tbats")
    res = tbats.runtbats(rpy2.robjects.FloatVector(single_x),rpy2.robjects.FloatVector([FORECASTING_STEPS]))
    pred = np.array(res[1])
    return evaluateARIMAprediction(pred,single_y,scaler)

def evaluateBAGGED_ETS(single_x,single_y,scaler):
    with open(BAGGED_ETS_FILE, 'r') as f:
        string = f.read()
    bagged_ets = STAP(string, "bagged_ets")
    res = bagged_ets.runbaggedets(rpy2.robjects.FloatVector(single_x),rpy2.robjects.FloatVector([FORECASTING_STEPS]))
    pred = np.array(res[7])
    #display(res[7])
    return evaluateARIMAprediction(pred,single_y,scaler)

In [4]:
df = pd.read_csv(DATASET_STATES_CSV,index_col=0,parse_dates=True,encoding = "utf-8")
df = normalize_df(df,DATA_SCALER())

(67, 16)

In [6]:
WINDOWSIZE = 6
INPUTSIZE = 6
pred= {}
overall_acc = 0
for state in df:
    pred[state] = []
    series = np.array(df[state].values)
    max_steps = int((len(series)-INPUTSIZE)/WINDOWSIZE)
    offset = (len(series)-INPUTSIZE)%WINDOWSIZE
    acc = 0
    #display(series)
    for i in range(0,max_steps):
        res = getSARIMAPred(series[:offset+(i*WINDOWSIZE)+INPUTSIZE])
        #res = getThetaPred(series[offset+i*WINDOWSIZE:offset+(i*WINDOWSIZE)+INPUTSIZE])
        pred[state].extend(res)
        display(series[:offset+(i*WINDOWSIZE)+INPUTSIZE])
        display(series[(offset+(i*WINDOWSIZE)+INPUTSIZE):(offset+(i*WINDOWSIZE)+INPUTSIZE+6)])
        display(res)
        #acc += evaluateARIMAprediction(series[(offset+(i*WINDOWSIZE)+INPUTSIZE):(offset+(i*WINDOWSIZE)+6)],res,None)
        #display(acc)
    #acc = acc/(max_steps)
    #display(acc)
    error = evaluateARIMAprediction(series[INPUTSIZE+offset:],pred[state],None)
    overall_acc += error
    display(series[INPUTSIZE+offset:])
    display(pred[state])
    display("StateError: "+str(sqrt(error)))
            
display(sqrt(overall_acc/16))
    

array([0.93467122, 0.89254199, 0.77782522, 0.64830629, 0.50128096,
       0.34365215, 0.43566752])

array([0.48861372, 0.32678622, 0.26256049, 0.29102619, 0.40140194,
       0.79721748])

array([0.59414313, 0.62522343, 0.64726281, 0.66289118, 0.67397342,
       0.68183196])

array([0.93467122, 0.89254199, 0.77782522, 0.64830629, 0.50128096,
       0.34365215, 0.43566752, 0.48861372, 0.32678622, 0.26256049,
       0.29102619, 0.40140194, 0.79721748])

array([0.83322659, 0.83589525, 0.7738756 , 0.60318816, 0.53497723,
       0.64005124])

...

array([ 0.00650142, -0.06295556, -0.09349661, -0.09466989, -0.0342159 ,
        0.18222451])

array([ 0.56094564,  0.4661298 ,  0.41566497,  0.40852973,  0.50820109,
        0.83702925,  0.89306016,  0.91350367,  0.87388974,  0.7417545 ,
        0.67509474,  0.68998697,  0.61696175,  0.51905199,  0.4396909 ,
        0.42146791,  0.46189602,  0.72826859,  0.75405613,  0.69007579,
        0.54184924,  0.41069102,  0.32933148,  0.33578577,  0.2656324 ,
        0.17120144,  0.11784995,  0.12359368,  0.22162186,  0.49933385,
        0.47853505,  0.39951445,  0.27455886,  0.17191201,  0.12069221,
        0.13998105,  0.1341337 ,  0.0568451 ,  0.0169351 ,  0.0143297 ,
        0.07952392,  0.31994612,  0.33784344,  0.26159107,  0.19362861,
        0.11149929,  0.05487624,  0.06362506,  0.08652594,  0.02365585,
        0.        ,  0.00190964,  0.06566793,  0.28564661,  0.28113157,
        0.24111795,  0.16569458,  0.07934628,  0.00790502,  0.00282745,
        0.02112447, -0.04112387, -0.06723709, -0.06298851, -0.00538844,
        0.19614519])

[0.692234965821072,
 0.8370663012324395,
 0.9613711276360395,
 1.0175793176149925,
 0.9876662941749157,
 0.8884762106506892,
 1.0964594844906346,
 1.229723361923716,
 1.2205049390902893,
 1.0913900069970373,
 0.8925462566235262,
 0.684927313613391,
 0.7074900058264865,
 0.721903844859303,
 0.7304991383616161,
 0.7331394012771627,
 0.7313455414197302,
 0.7272641364928506,
 0.7819200183871095,
 0.8235277392335373,
 0.8189565209186614,
 0.7291516828798842,
 0.6890444382391797,
 0.7110660894378019,
 0.29533755370437753,
 0.2518287841255182,
 0.23406505494354046,
 0.2727191851008234,
 0.36032098313175065,
 0.6442376074285808,
 0.5337082467527372,
 0.521761598449057,
 0.44631971567033135,
 0.31669814174202887,
 0.24626701726662306,
 0.25727481384707573,
 0.08562065008763423,
 -0.006089152834333533,
 -0.06508721512185507,
 -0.0709799951143488,
 0.005194172621664439,
 0.28104599908787087,
 0.31378986648045115,
 0.2674371696120816,
 0.17003262293399365,
 0.04802711956394051,
 -0.016216156719472

'StateError: 0.19746051532538483'

0.20105712431626716

In [7]:
display(len(pred[list(pred.keys())[0]]))

66

In [8]:
df_ARIMA = pd.DataFrame.from_dict(pred)
df_ARIMAIMA.to_csv('SARIMA_Pred.csv')

In [6]:
Final_Results = {}
for rep in range(0,1):
    overall_test_scores = {}
    nbr_of_states = len(list(df))
    approaches = []
    approaches.append({'name':'STL_seas_naive','sc':None, 'func': evaluateSTLSeasNaive,'wsc':False})
    approaches.append({'name':'ARMA','sc':None, 'func': evaluateARMA,'wsc':False})
    approaches.append({'name':'ARIMA','sc':None, 'func': evaluateARIMA,'wsc':False})
    approaches.append({'name':'SARIMA','sc':None, 'func': evaluateSARIMA,'wsc':False})
    approaches.append({'name':'Theta','sc':None, 'func': evaluateTheta,'wsc':False})
    approaches.append({'name':'4Theta','sc':None, 'func': evaluateFourTheta,'wsc':False})
    approaches.append({'name':'ETS','sc':None, 'func': evaluateETS,'wsc':False})
    approaches.append({'name':'BAGGED_ETS','sc':None, 'func': evaluateBAGGED_ETS,'wsc':False})
    approaches.append({'name':'TBATS','sc':None, 'func': evaluateTBATS,'wsc':False})
    start = time.time()
    for state in df:
        print(f'{state}:')
        for approach in approaches:
            scaler = approach['sc']
            if scaler == DATA_SCALER and not approach['wsc'] :#already scaled like that
                scaler = None
            series = []
            if not approach['wsc'] and not scaler == None: #not windowscaled and have a scaler
                scaler = scaler()
                series = normalizeSeries(df[state],scaler)
            else:#windowscaled or no scaler
                series = df[state]
                if not scaler == None:#init scaler if windowscaled
                    scaler = scaler()
            x,y = getTrainTestSplit(series)
            test_score = approach['func'](x,y,scaler)
            prevStateDict = Final_Results.get(state,{})
            prevRes = prevStateDict.get(approach['name'],[])
            prevRes.append(test_score)
            prevStateDict[approach['name']] = prevRes
            Final_Results[state] = prevStateDict
            print(approach['name']+':'+ str(sqrt(test_score)))
            overall_test_scores[approach['name']]= overall_test_scores.get(approach['name'],0) + test_score
        print('')
    for approach in approaches:
        overall_test_scores[approach['name']] = overall_test_scores[approach['name']]/nbr_of_states
        print(approach['name']+':'+str(sqrt(overall_test_scores[approach['name']])))
    end = time.time()
    print(f'{(end - start)/60} minutes runtime')

with open('my_Final_Res_St.json', 'w') as f:
    json.dump(Final_Results, f)

Schleswig-Holstein:
STL_seas_naive:0.1886240854475677
ARMA:0.21641680408873454
ARIMA:0.21641680408873454
SARIMA:0.09558240566347684
Theta:0.18979184475725427
4Theta:0.1902357148999786
ETS:0.1859510498008255
BAGGED_ETS:0.193425049481766
TBATS:0.20467222124069703

Freie und Hansestadt Hamburg:
STL_seas_naive:0.14448850976433603
ARMA:0.10392128131754881
ARIMA:0.10392128131754881
SARIMA:0.13400950800557324
Theta:0.13744678059755916
4Theta:0.13657192400509438
ETS:0.1410588857027093
BAGGED_ETS:0.11496170182816014
TBATS:0.1606272642643329

Niedersachsen:
STL_seas_naive:0.11772631451223484
ARMA:0.1785692339330329
ARIMA:0.1785692339330329
SARIMA:0.03543501268976395
Theta:0.13013461697061954
4Theta:0.12978436156076656
ETS:0.13302653066120448
BAGGED_ETS:0.11687879735145094
TBATS:0.20104652826534375

Freie Hansestadt Bremen:
STL_seas_naive:0.19276117666091125
ARMA:0.27754696830859843
ARIMA:0.27754696830859843
SARIMA:0.10235375756002674
Theta:0.280434029692953
4Theta:0.2797878931047171
ETS:0.283352